## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [3]:
bike_df = pd.read_csv('./bikeshare.csv')

In [4]:
bike_df.shape

(10886, 12)

In [5]:
bike_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [6]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datetime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [7]:
# 현재 datetime 변수는 문자열로 입력되어 있으므로, 날짜형 변수로 변환해주는 것이 필요함.

bike_df['datetime'] = bike_df['datetime'].apply(pd.to_datetime)

- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

In [8]:
bike_df["year"] = bike_df['datetime'].map(lambda x: x.year)
bike_df["month"] = bike_df['datetime'].map(lambda x: x.month)
bike_df["day"] = bike_df['datetime'].map(lambda x: x.day)
bike_df["hour"] = bike_df['datetime'].map(lambda x: x.hour)

# 시간이 시, 분, 초를 포함하는 time인 줄 알고 처음에 헤맴;;

- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [9]:
bike_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2


* datetime, casual, registered 칼럼을 삭제 하시오.(casual + registered = count이므로 따로 필요하지 않습니다.) 

In [10]:
bike_df.drop(['datetime', 'casual', 'registered'], axis=1, inplace=True)

In [11]:
bike_df.head(3)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2


## 2. 선형 회귀 실습하기

### (1) X_features와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. (종속변수 = count)

In [12]:
X_features = bike_df.drop(['count'], axis=1) 
y_target = bike_df['count']

In [13]:
X_features.head(3)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,2


### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [14]:
X_features_ohe = pd.get_dummies(X_features, columns=['year'])
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['month'])
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['day'])
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['hour'])
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['holiday'], drop_first=True)    # 이진 분류
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['workingday'], drop_first=True) # 이진 분류
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['season'])
X_features_ohe = pd.get_dummies(X_features_ohe, columns=['weather'])

In [15]:
X_features_ohe

,temp,atemp,humidity,windspeed,year_2011,year_2012,month_1,month_2,month_3,month_4,...,holiday_1,workingday_1,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4
0,9.84,14.395,81,0.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,9.02,13.635,80,0.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,9.02,13.635,80,0.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,9.84,14.395,75,0.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,9.84,14.395,75,0.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,15.58,19.695,50,26.0027,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
10882,14.76,17.425,57,15.0013,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
10883,13.94,15.910,61,15.0013,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
10884,13.94,17.425,61,6.0032,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0


### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features_ohe, y_target, test_size=0.3, random_state=0)

### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)

pred=lin_reg.predict(X_test)

In [30]:
mse=mean_squared_error(y_test, pred)

print('lin_reg의 RMSE: ', np.sqrt(mse))

lin_reg의 RMSE:  101.62042299204896


In [21]:
lin_reg.intercept_, lin_reg.coef_

(6951111714170.16,
 array([ 2.76786075e+00,  1.85835885e+00, -8.03537286e-01, -7.33259308e-01,
        -8.42620610e+12, -8.42620610e+12, -7.77285795e+11, -7.77285795e+11,
        -7.77285795e+11,  7.73283443e+11,  7.73283443e+11,  7.73283443e+11,
         3.12916929e+12,  3.12916929e+12,  3.12916929e+12, -7.54545545e+11,
        -7.54545545e+11, -7.54545545e+11, -5.46101466e+10, -5.46101466e+10,
        -5.46101466e+10, -5.46101466e+10, -5.46101466e+10, -5.46101466e+10,
        -5.46101466e+10, -5.46101466e+10, -5.46101466e+10, -5.46101466e+10,
        -5.46101466e+10, -5.46101466e+10, -5.46101466e+10, -5.46101466e+10,
        -5.46101466e+10, -5.46101466e+10, -5.46101466e+10, -5.46101466e+10,
        -5.46101466e+10,  1.60084506e+11,  1.60084506e+11,  1.60084506e+11,
         1.60084506e+11,  1.60084506e+11,  1.60084506e+11,  1.60084506e+11,
         1.60084506e+11,  1.60084506e+11,  1.60084506e+11,  1.60084506e+11,
         1.60084506e+11,  1.60084506e+11,  1.60084506e+11,  1.6008450

#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000, tol=1e-3, penalty=None, eta0=0.1 인 옵션을 추가하시오.

In [22]:
from sklearn.linear_model import SGDRegressor

sgd_reg=SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X_train, y_train)

pred_sgd=sgd_reg.predict(X_test)

In [28]:
mse1=mean_squared_error(y_test, pred_sgd)
mse11=mean_squared_error(y_test, pred_sgd, squared=False) # mean_square_error에 rmse 출력 옵션 존재!

print('sgd_reg의 RMSE: ', np.sqrt(mse1))
print('sgd_reg의 RMSE: ', mse11) # same! 그나저나 이렇게 큰 값이 나오는 게 맞나..?

sgd_reg의 RMSE:  9295609883801.3
sgd_reg의 RMSE:  9295609883801.3


In [29]:
sgd_reg.intercept_, sgd_reg.coef_

(array([-1.4754445e+11]),
 array([ 2.98548782e+11, -1.18236600e+10,  4.46224466e+10,  5.80450420e+10,
        -1.26303339e+11, -2.12411106e+10,  3.52382875e+10, -1.26257324e+11,
         8.06317959e+10, -1.01310832e+11,  1.14018689e+11, -1.23253966e+11,
         2.85008962e+10,  6.91004040e+10, -1.15955741e+11, -2.58012608e+11,
         2.85941578e+10,  2.21161790e+11, -2.79389455e+11,  1.17780120e+11,
         5.09491209e+10, -9.09070586e+10,  2.36633692e+11,  4.73635777e+10,
        -7.63669030e+09, -1.31295092e+11, -1.48528097e+11, -1.11389657e+11,
        -1.69947603e+11,  1.48861213e+11,  3.87028769e+09,  1.52146711e+10,
         6.16739514e+10,  3.47682176e+10,  4.56779525e+10,  9.58451964e+09,
         1.91718801e+10,  1.02337161e+10,  1.57740667e+11,  6.75307778e+10,
         2.00980208e+11, -1.57273383e+11, -4.00693796e+11,  8.55050599e+10,
         4.29989712e+10, -1.77114455e+11, -1.60669742e+10,  6.03018895e+10,
        -4.95424738e+10,  1.20516521e+11,  9.12822383e+10, -1.

#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [31]:
from sklearn.linear_model import Ridge, Lasso
ridge_reg=Ridge(alpha=0.1, solver='cholesky')
ridge_reg.fit(X_train, y_train)

pred_ridge = ridge_reg.predict(X_test)

In [34]:
rmse2=mean_squared_error(y_test, pred_ridge, squared=False)
print('ridge_reg의 RMSE: ', rmse2)

ridge_reg의 RMSE:  101.61963942908895


In [37]:
ridge_reg.intercept_, ridge_reg.coef_

(93.6445708422628,
 array([ 2.77634352e+00,  1.85639233e+00, -8.04848801e-01, -7.32755485e-01,
        -4.30350494e+01,  4.30350494e+01, -2.78810473e+01, -1.12490664e+01,
         6.11833388e+00, -9.97388340e+00,  1.52190575e+01,  2.91908549e+00,
        -1.08898141e+01, -1.00813596e+01,  2.46161787e+01,  2.21827790e+01,
         1.88005091e-01, -1.16826893e+00, -9.80461367e+00, -2.81920545e+00,
         1.30082430e+00,  8.68180181e-01,  2.18863534e+00,  4.24369489e-01,
        -7.62086672e+00, -9.53378637e+00,  7.17799431e-02,  1.02672261e+00,
         2.07625639e+00, -5.06886750e+00,  2.13603540e+00,  2.81141108e+00,
         6.74382602e+00,  2.20609097e+00,  1.81487741e+01, -6.33766138e+00,
         1.18209526e+00, -1.24689442e+02, -1.45447920e+02, -1.59260542e+02,
        -1.66295512e+02, -1.68110470e+02, -1.49564143e+02, -9.42789028e+01,
         4.05901994e+01,  1.92968025e+02,  4.08782913e+01, -1.66084953e+01,
         5.55253182e+00,  5.22344983e+01,  4.69859826e+01,  3.2745481

In [35]:
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_train, y_train)

pred_lasso=lasso_reg.predict(X_test)

In [36]:
rmse3=mean_squared_error(y_test, pred_lasso, squared=False)

print('lasso_reg의 RMSE: ', rmse3)

lasso_reg의 RMSE:  101.56225421214636


In [38]:
lasso_reg.intercept_, lasso_reg.coef_

(215.9984202149896,
 array([ 3.27180766e+00,  1.76978059e+00, -8.73804775e-01, -7.10780296e-01,
        -8.50381823e+01,  0.00000000e+00, -2.96980105e+01, -1.35728022e+01,
         0.00000000e+00, -4.87987262e+00,  1.71401016e+01,  2.85040566e+00,
        -5.27072118e+00, -3.75178267e+00,  3.03745171e+01,  1.90334989e+01,
         0.00000000e+00, -0.00000000e+00, -8.66790918e+00, -1.84686122e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.39095552e+00, -8.72523168e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.85360130e+00,  0.00000000e+00,  1.87183492e-01,
         3.68292429e+00,  0.00000000e+00,  1.48958750e+01, -4.98174542e+00,
         0.00000000e+00, -1.49473706e+02, -1.70097066e+02, -1.83473824e+02,
        -1.90307400e+02, -1.91967177e+02, -1.73459149e+02, -1.18167091e+02,
         1.18118347e+01,  1.63673667e+02,  1.08736427e+01, -4.25399488e+01,
        -2.08513802e+01,  2.03154895e+01,  1.47011803e+01,  1.274470

## 3. 로지스틱 회귀 분석 실습 _ 와인 종류 분류하기 : 와인의 화학 조성을 사용하여 와인의 종류를 예측하기 위한 데이터이다. 

In [26]:
# 와인 데이터를 불러오기 위한 코드입니다. 꼭 실행해주세요.
from sklearn import datasets

ㅇ 타겟 데이터 : 와인의 종류 0, 1, 2의 세가지 값

ㅇ 특징 데이터

* 알콜(Alcohol)

* 말산(Malic acid)

* 회분(Ash)

* 회분의 알칼리도(Alcalinity of ash)

* 마그네슘(Magnesium)

* 총 폴리페놀(Total phenols)

* 플라보노이드 폴리페놀(Flavanoids)

* 비 플라보노이드 폴리페놀(Nonflavanoid phenols)

* 프로안토시아닌(Proanthocyanins)

* 색상의 강도(Color intensity)

* 색상(Hue)

* 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)

* 프롤린(Proline)

####3-1. 와인 데이터 가져오기 

In [27]:
wine_data = datasets.load_wine()

더 자세한 데이터에 대한 설명을 보고싶으시다면 아래의 코드를 실행해주시면 됩니다. 

In [28]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

#### 3-2. 입력변수와 목표변수 설정하기
* 입력변수를 사용하기 편하게 pandas dataframe으로 변환해주세요.
  - 입력변수는 data (.data) 함수를 써서 가져오시면 됩니다.
  - 칼럼명 가져오기 : .feature_names 
* 목표변수를 사용하기 편하게 pandas dataframe으로 변환해주세요
  - 목표변수는 target (.target) 함수를 써서 가져요오시면 됩니다. 
  - 칼럼명은 'target'으로 지정해주세요

In [29]:
# 입력 변수를 사용하기 편하게 pandas dataframe으로 변환
X = pd.DataFrame(data=wine_data.data, columns=[wine_data.feature_names])
# 목표 변수를 사용하기 편하게 pandas dataframe으로 변환
y = pd.DataFrame(data=wine_data.target, columns=['target'])

In [30]:
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [31]:
y.head()

,target
0,0
1,0
2,0
3,0
4,0


#### 3-3. 훈련 데이터, 테스트 데이터 만들기
* train_test_split 함수를 사용해 테스트 셋의 비율이 0.2가 되도록 X_train, X_test, y_train, y_test 을 만들어주세요. random_state=5로 설정해주세요.
* 경고메세지를 없애도록 y_train = y_train.values.ravel() 를 실행해주세요

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

y_train = y_train.values.ravel()

#### 3-4. 모델훈련
* 로지스틱회귀 함수에서 옵션을 solver='saga', max_iter=7500으로 설정해주세요.
* 학습데이터로 모델을 학습시켜주세요

In [33]:
# 필요한 라이브러리 import
from sklearn.linear_model import LogisticRegression
logit_reg=LogisticRegression(max_iter=7500, solver='saga')
logit_reg.fit(X_train, y_train)

LogisticRegression(max_iter=7500, solver='saga')

#### 3-5. 예측
* 테스트 데이터 셋으로 로지스틱 회귀 모델을 이용해 와인 데이터 분류를 예측해 결과를 y_test_predict 변수에 담아주세요
* score함수를 사용해 로지스틱 회귀 모델의 성능을 score 변수에 담아주세요
* y_test_predict와 score 결과를 확인해주세요.

In [34]:
# 로지스틱 회귀 모델를 이용해서 각 와인 데이터 분류를 예측함
y_test_predict = logit_reg.predict(X_test)

# 로지스틱 회귀 모델의 성능 확인 (정확성 %를 리턴함)
score = logit_reg.score(X_train, y_train)  # 지난 과제에서와 같이 단순 정확도!
# 결과확인 
y_test_predict, score

(array([0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2]),
 0.9436619718309859)

#### (선택) 3-6. GridSearchCV 복습

`LogisticRegression` 모델에 `param_grid`를 사용하여 GridSearchCV를 진행해주세요.
- cv = 5
- 추가적으로 코드를 작성하실 필요 없이, 골뱅이@@@ 부분의 코드를 완성해서 출력해주시면 됩니다. 

In [36]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

# param_grid (수정하셔도 됩니다)
params = {
    'penalty':['l1', 'l2', 'elasticnet'],        # l1 is Lasso, l2 is Ridge
    'C': np.linspace(0.00002,1,100)
}

# GridSearch
grid_search = GridSearchCV(logit_reg, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=7500, solver='saga'),
             param_grid={'C': array([2.00000000e-05, 1.01208081e-02, 2.02216162e-02, 3.03224242e-02,
       4.04232323e-02, 5.05240404e-02, 6.06248485e-02, 7.07256566e-02,
       8.08264646e-02, 9.09272727e-02, 1.01028081e-01, 1.11128889e-01,
       1.21229697e-01, 1.31330505e-01, 1.41431313e-01, 1.51532121e-01,
       1.61632929e...
       8.08084646e-01, 8.18185455e-01, 8.28286263e-01, 8.38387071e-01,
       8.48487879e-01, 8.58588687e-01, 8.68689495e-01, 8.78790303e-01,
       8.88891111e-01, 8.98991919e-01, 9.09092727e-01, 9.19193535e-01,
       9.29294343e-01, 9.39395152e-01, 9.49495960e-01, 9.59596768e-01,
       9.69697576e-01, 9.79798384e-01, 9.89899192e-01, 1.00000000e+00]),
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

학습된 gridsearch 모델에 저장되어 있는 best parameter, best score 를 출력해주세요.

In [37]:
grid_search.best_params_, grid_search.best_score_

({'C': 0.08082646464646465, 'penalty': 'l2'}, 0.9302955665024631)